In [53]:
import cv2
from ultralytics import YOLO
import numpy as np
from tqdm import tqdm
import torch
import matplotlib.pyplot as plt

In [54]:
# 프레임이 아닌 비디오를 모델의 입력으로...
model = YOLO('../model/yolov8m-face.pt')
videoPath = 'testvideoShorts.mp4'

results = model(videoPath, stream=True ,verbose=False,show_labels=False ,device='mps')

In [ ]:

for result in results:
    ori = result.orig_img
    for box in result.boxes:
        left_x = int(box.xyxy.tolist()[0][0])
        left_y = int(box.xyxy.tolist()[0][1])
        right_x = int(box.xyxy.tolist()[0][2])
        right_y = int(box.xyxy.tolist()[0][3])
        
        center_x = (left_x + right_x) // 2
        center_y = (left_y + right_y) // 2
        axis_major = (right_x - left_x) // 2
        axis_minor = (right_y - left_y) // 2
        face = ori[left_y:right_y, left_x:right_x]
        
        ori[left_y:right_y, left_x:right_x] = cv2.blur(face, (40, 40))
        
        maskShape = (ori.shape[0], ori.shape[1], 1)
        mask = np.full(maskShape, 0, dtype=np.uint8)
        cv2.ellipse(mask, (center_x, center_y), (axis_major, axis_minor), 0, 0, 360, (255, 255, 255), -1)
        mask_inv = cv2.bitwise_not(mask)
        img1_bg = cv2.bitwise_and(result.orig_img,result.orig_img,mask = mask_inv)
        img2_fg = cv2.bitwise_and(ori,ori,mask = mask)
        dst = cv2.add(img1_bg,img2_fg)
        ori = dst

    ori = cv2.cvtColor(dst, cv2.COLOR_BGR2RGB)
    plt.imshow(ori)
    break


In [ ]:


for result in results:
    ori = result.orig_img
    for box in result.boxes:
        left_x = int(box.xyxy.tolist()[0][0])
        left_y = int(box.xyxy.tolist()[0][1])
        right_x = int(box.xyxy.tolist()[0][2])
        right_y = int(box.xyxy.tolist()[0][3])
        
        center_x = (left_x + right_x) // 2
        center_y = (left_y + right_y) // 2
        axis_major = (right_x - left_x) // 2
        axis_minor = (right_y - left_y) // 2
        face = ori[left_y:right_y, left_x:right_x]
        
        mask = np.zeros_like(ori)
        cv2.ellipse(mask, (center_x, center_y), (axis_major, axis_minor), 0, 0, 360, (255, 255, 255), -1)
        inverse_mask = cv2.bitwise_not(mask)
        face_with_ellipse = cv2.bitwise_and(ori, mask)
        result = cv2.add(ori, mask)
        
        # ori[left_y:right_y, left_x:right_x] = cv2.blur(result, (40, 40))
        # ori = result
    ori = cv2.cvtColor(face_with_ellipse, cv2.COLOR_BGR2RGB)
    plt.imshow(ori)
    break
    

In [ ]:


for result in results:
    ori = result.orig_img
    for box in result.boxes:
        left_x = int(box.xyxy.tolist()[0][0])
        left_y = int(box.xyxy.tolist()[0][1])
        right_x = int(box.xyxy.tolist()[0][2])
        right_y = int(box.xyxy.tolist()[0][3])
        
        # 타원형 중심과 축 길이 계산
        center_x = (left_x + right_x) // 2
        center_y = (left_y + right_y) // 2
        axis_major = (right_x - left_x) // 2
        axis_minor = (right_y - left_y) // 2
        
        # 타원형 마스크 생성
        mask = np.zeros_like(ori)
        cv2.ellipse(mask, (center_x, center_y), (axis_major, axis_minor), 0, 0, 360, (255, 255, 255), -1)
        
        # 원본 이미지 * 타원형 영역
        face_with_ellipse = cv2.bitwise_and(ori, mask)
        blurred_face = cv2.blur(face_with_ellipse, (40, 40))
        
        # !(타원형 마스크)
        inverse_mask = cv2.bitwise_not(mask)
        
        # 원본 이미지에서 타원형 영역을 제외한 나머지 부분 추출
        background = cv2.bitwise_and(ori, inverse_mask)
        result_image = cv2.add(blurred_face, background)

        
        # ori[left_y:right_y, left_x:right_x] = cv2.blur(result, (40, 40))
        ori = result_image
    ori = cv2.cvtColor(ori, cv2.COLOR_BGR2RGB)
    plt.imshow(ori)
    break
    

In [ ]:
cv2.BORDER_CONSTANT

In [15]:
# if (torch.backends.mps.is_available()):
#     mps_device = torch.device("mps")
#     print("mps is useable")


mps is useable


In [18]:
# # 프레임이 아닌 비디오를 모델의 입력으로...
# model = YOLO('../model/yolov8m-face.pt')
# model.to(mps_device)

# videoPath = 'testvideoShorts.mp4'

# results = model(videoPath, stream=True ,verbose=False,show_labels=False ,device='mps')


# cap = cv2.VideoCapture(videoPath)
# fps = round(cap.get(cv2.CAP_PROP_FPS))
# w = round(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
# h = round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
# fourcc = cv2.VideoWriter_fourcc(*'DIVX')
# saveVideo = cv2.VideoWriter('outputShortmps.mp4', fourcc, fps, (w, h))
# frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)

# cap.release()

# for result in tqdm(results,  total=frame_count):
#     ori = result.orig_img
#     for box in result.boxes:
#                 left_x = int(box.xyxy.tolist()[0][0])
#                 left_y = int(box.xyxy.tolist()[0][1])
#                 right_x = int(box.xyxy.tolist()[0][2])
#                 right_y = int(box.xyxy.tolist()[0][3])
                
#                 face = ori[left_y:right_y, left_x:right_x]
                
#                 ori[left_y:right_y, left_x:right_x] = cv2.blur(face, (40, 40))
    
#     saveVideo.write(ori)


# saveVideo.release() #1분 46초

OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
100%|██████████| 1674/1674.0 [01:44<00:00, 15.98it/s]


In [5]:
# 프레임이 아닌 비디오를 모델의 입력으로...
model = YOLO('../model/yolov8m-face.pt')
videoPath = 'testvideoShorts.mp4'

results = model(videoPath, stream=True ,verbose=False,show_labels=False ,device='mps')


cap = cv2.VideoCapture(videoPath)
fps = round(cap.get(cv2.CAP_PROP_FPS))
w = round(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
saveVideo = cv2.VideoWriter('outputShorts.mp4', fourcc, fps, (w, h))
frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)

cap.release()

for result in tqdm(results,  total=frame_count):
    ori = result.orig_img
    for box in result.boxes:
                left_x = int(box.xyxy.tolist()[0][0])
                left_y = int(box.xyxy.tolist()[0][1])
                right_x = int(box.xyxy.tolist()[0][2])
                right_y = int(box.xyxy.tolist()[0][3])
                
                face = ori[left_y:right_y, left_x:right_x]
                
                ori[left_y:right_y, left_x:right_x] = cv2.blur(face, (40, 40))
    
    saveVideo.write(ori)


saveVideo.release() #1분 45초

OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
100%|██████████| 1674/1674.0 [01:44<00:00, 16.06it/s]


In [ ]:
# model = YOLO('../model/yolov8m-face.pt')
# cap = cv2.VideoCapture('testvideoShorts.mp4')
# fps = round(cap.get(cv2.CAP_PROP_FPS))
# w = round(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
# h = round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
# fourcc = cv2.VideoWriter_fourcc(*'DIVX')
# saveVideo = cv2.VideoWriter('outputShort.mp4', fourcc, fps, (w, h))

# for i in tqdm(range(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))):
# # while(True):
#     retval, frame = cap.read()
    
#     if not(retval):	# 프레임정보를 정상적으로 읽지 못하면
#         break  # while문을 빠져나가기
    
#     else:
#         results = model(frame, verbose=False)
#         boxes = results[0].boxes
        
#         for box in boxes:
#             left_x = int(box.xyxy.tolist()[0][0])
#             left_y = int(box.xyxy.tolist()[0][1])
#             right_x = int(box.xyxy.tolist()[0][2])
#             right_y = int(box.xyxy.tolist()[0][3])
            
#             face = frame[left_y:right_y, left_x:right_x]
            
#             frame[left_y:right_y, left_x:right_x] = cv2.blur(face, (40, 40))
            
#         saveVideo.write(frame)
            
#         if cv2.waitKey(1) == ord('q'):
#             break

    
# if cap.isOpened():	# 영상 파일(카메라)이 정상적으로 열렸는지(초기화되었는지) 여부
#     cap.release()	# 영상 파일(카메라) 사용을 종료
#     saveVideo.release()
    
# cv2.destroyAllWindows() 
# cv2.waitKey(1) #맥에서 창 종료가 안되는 문제 해결
# cv2.waitKey(1)
# cv2.waitKey(1)
# cv2.waitKey(1)